In [1]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
from PyPDF2 import PdfFileWriter, PdfFileReader

In [8]:
def isnan(value):
    try:
        return math.isnan(float(value))
    except:
        return False
    
def icici_p3(f):
    tables=camelot.read_pdf(f, flavor="stream", pages="all")
    if len(tables) != 0:
        dt = pd.DataFrame(); tmp = pd.DataFrame();
        for i in range (len(tables)):
            tmp = tables[i].df
            if (tmp.shape[1] < 3):
                print("\nDiiferent structure on page:",i); pass
            elif (tmp.shape[1] == 6):
                tmp = pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex = True)
                dt = pd.concat([dt,tmp]).reset_index(drop=True)
            else:
                print("\nAnother Structure on page:",i); pass
            
        df = dt
        try:
            idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
            df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)
            print("1st attempt")
        except:
            try:
                idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
            except:
                print("\nAxis-Column headers missing"); pass
            
        drop_idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index]
        print(drop_idx)
        print(len(drop_idx))
        if len(drop_idx)>0:
            df.drop(df.index[drop_idx],inplace=True)
        df.to_csv("check001.csv", index=False)
        
        try:
            bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
        except: print("\nBalance columns missing")
            
        try:
            dat=[c for c in df.columns if "TRANSACTION DATE" in str(c).upper() ][0]
        except:
            try:
                dat=[c for c in df.columns if "TXN DATE" in str(c).upper() ][0]
            except:
                try:
                    dat=[c for c in df.columns if "DATE" in str(c).upper() ][0]
                except:pass
                
        try:
            chq=[c for c in df.columns if "CHQ" in str(c).upper() ][0]
        except:
            try:
                chq=[c for c in df.columns if "CHEQUE" in str(c).upper() ][0]
            except:
                try:
                    df["CHQ"] = np.nan
                    chq=[c for c in df.columns if "CHQ" in str(c).upper() ][0]
                except:pass
                
        try:
            narr=[c for c in df.columns if "REMARKS" in str(c).upper() ][0]
        except:
            try:
                narr=[c for c in df.columns if "PARTICULARS" in str(c).upper() ][0]
            except:
                try:
                    narr=[c for c in df.columns if "DESCRIPTION" in str(c).upper() ][0]
                except:
                    try:
                        narr=[c for c in df.columns if "DETAILS" in str(c).upper() ][0]
                    except:
                        try:
                            narr=[c for c in df.columns if "NARRATION" in str(c).upper() ][0]
                        except:pass
        
        try:
            for j,i in enumerate(df[narr]):
                if isnan(i) == True:
                    df[narr][j] = df[narr][j-1]+df[narr][j+1]
                else:
                    pass
        except:pass
        
        df = df.replace(r'^\s*$', np.nan, regex=True)
        df = df[ df.isnull().sum(axis=1) < df.shape[1]-2].reset_index(drop=True)
    
        try:
            wdl=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
        except:
            try:
                wdl=[c for c in df.columns if "DEBIT" in str(c).upper() ][0]
            except: pass 

        try:
            dep=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
        except:
            try:
                dep=[c for c in df.columns if "CREDIT" in str(c).upper() ][0]
            except: pass
            
        df[dep]=df[dep].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
        df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
        df[wdl]=df[wdl].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float) *-1
        df[dep]=df[dep].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)
        df[bal]=df[bal].astype(str).apply(lambda x:str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)


        df = df[[dat,chq,narr,wdl,dep,bal]]
        df.columns = ["Xns Date","Cheque No","Narration","Debits","Credits","Balance"]

        df["Xns Date"] = df["Xns Date"].str.replace("\r","")

        df.to_csv("check02.csv", index=False)    
        return df
    
f = r"C:\Users\rites\Desktop\mudracircle\bks_raw\Parsing_testing\ICICI\ICICI\pattern_3\icici01.pdf"
df = icici_p3(f)
print("Parsed")


Diiferent structure on page: 6
1st attempt
[44, 102, 171, 240, 318]
5
Parsed


c:\users\rites\appdata\local\programs\python\python36-32\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
c:\users\rites\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\rites\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand